In [ ]:
!pip install SimpleITK vtk pydicom ipympl itkwidgets numpy-stl -q

In [ ]:
!git clone https://github.com/dave3d/dicom2stl.git
!wget https://raw.githubusercontent.com/InsightSoftwareConsortium/SimpleITK-Notebooks/master/Python/gui.py -O gui.py

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import itkwidgets
import SimpleITK as sitk
import glob
import re
from gui import MultiImageDisplay
sys.path.append('dicom2stl')
from utils import dicomutils
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [ ]:
DS_PATH = r'../input/rsna-miccai-brain-tumor-radiogenomic-classification'
def make_image(patient_id, split='train'):
    path_flair = os.path.join(DS_PATH, split, patient_id, 'FLAIR')
    path_t1w = os.path.join(DS_PATH, split, patient_id, 'T1w')
    path_t1wce = os.path.join(DS_PATH, split, patient_id, 'T1wCE')
    path_t2w = os.path.join(DS_PATH, split, patient_id, 'T2w')
    
    flair_dicom, flair_modality = dicomutils.loadLargestSeries(path_flair)
    t1w_dicom, t1w_modality = dicomutils.loadLargestSeries(path_t1w)
    t1wce_dicom, t1wce_modality = dicomutils.loadLargestSeries(path_t1wce)
    t2w_dicom, t2w_modality = dicomutils.loadLargestSeries(path_t2w)
    return flair_dicom, t1w_dicom, t1wce_dicom, t2w_dicom

flair_dicom, t1w_dicom, t1wce_dicom, t2w_dicom = make_image('00003')

In [ ]:
%matplotlib widget
MultiImageDisplay(image_list=[flair_dicom, t1w_dicom], 
                  title_list=['FLAIR', 'T1w'])

In [ ]:
%matplotlib widget
MultiImageDisplay(image_list=[t1wce_dicom, t2w_dicom], 
                  title_list=['T1wCE', 'T2w'])

In [ ]:
train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
train_df['BraTS21ID5'] = train_df.BraTS21ID.apply(lambda x: format(x, '05d'))

test_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
test_df['BraTS21ID5'] = test_df.BraTS21ID.apply(lambda x: format(x, '05d'))

In [ ]:
train_df['scan'] = [['FLAIR', 'T1w', 'T1wCE', 'T2w']] * len(train_df)
train_df = train_df.explode('scan').reset_index(drop=True)

In [ ]:
test_df['scan'] = [['FLAIR', 'T1w', 'T1wCE', 'T2w']] * len(test_df)
test_df = test_df.explode('scan').reset_index(drop=True)

In [ ]:
OUTPUT_DIR = './mesh'
def save_point_cloud(patient_id, scan, split='train'):
    output = os.path.join(OUTPUT_DIR, patient_id)
    os.makedirs(output, exist_ok=True)
    output = os.path.join(output, scan)
    path = os.path.join(DS_PATH, split, patient_id, scan)
    os.system('python dicom2stl/dicom2stl.py -i 400 -o {output}.stl {path}'.format(output=output, path=path))


process_map(save_point_cloud, train_df.BraTS21ID5.values, train_df.scan.values)
process_map(save_point_cloud, test_df.BraTS21ID5.values, test_df.scan.values, ['test']*len(test_df))

# for patient in tqdm(train_df.BraTS21ID5.values):
#     save_point_cloud(patient, 'FLAIR')
#     save_point_cloud(patient, 'T1w')
#     save_point_cloud(patient, 'T1wCE')
#     save_point_cloud(patient, 'T2w')
    

In [ ]:
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot

# Create a new plot
figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

# Load the STL files and add the vectors to the plot
your_mesh = mesh.Mesh.from_file('./mesh/00000/T1w.stl')
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

# Auto scale to the mesh size
scale = your_mesh.points.flatten()
axes.auto_scale_xyz(scale, scale, scale)

# Show the plot to the screen
pyplot.show()


In [ ]:
!rm -r dicom2stl gui.py